In [37]:
%store -r model_data_a
%store -r model_data_b
%store -r target_repository_uri_a
%store -r target_repository_uri_b

In [38]:
image_uri_a = f"{target_repository_uri_a}:1"
image_uri_b = f"{target_repository_uri_b}:1"

In [39]:
container1 = { 
    'Image': image_uri_a,
    'ContainerHostname': 'containerA',
    'ModelDataUrl': model_data_a
}

container2 = { 
    'Image': image_uri_b,
    'ContainerHostname': 'containerB',
    'ModelDataUrl': model_data_b
}

In [40]:
model_name = "ab-testing"
endpoint_config_name = 'ab-testing-config'
endpoint_name = 'ab-testing-endpoint'

In [41]:
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.Session().client('sagemaker')

In [42]:
model_name_a = "ab-model-a"
model_name_b = "ab-model-b"
endpoint_config_name = 'ab-endpoint-config'
endpoint_name = 'ab-endpoint'

In [43]:
try:
    sm_client.delete_model(ModelName=model_name_a)
    sm_client.delete_model(ModelName=model_name_b)
except:
    pass

In [44]:
response = sm_client.create_model(
    ModelName        = model_name_a,
    ExecutionRoleArn = role,
    Containers       = [container1])
print(response)

response = sm_client.create_model(
    ModelName        = model_name_b,
    ExecutionRoleArn = role,
    Containers       = [container2])
print(response)

{'ModelArn': 'arn:aws:sagemaker:us-east-1:581320662326:model/ab-model-a', 'ResponseMetadata': {'RequestId': 'd6442c21-0202-4bee-9fca-0070aa6d98dc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd6442c21-0202-4bee-9fca-0070aa6d98dc', 'content-type': 'application/x-amz-json-1.1', 'content-length': '72', 'date': 'Mon, 03 May 2021 15:14:58 GMT'}, 'RetryAttempts': 0}}
{'ModelArn': 'arn:aws:sagemaker:us-east-1:581320662326:model/ab-model-b', 'ResponseMetadata': {'RequestId': 'f942a624-a026-4fe7-86d2-dd4d5a907e6f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f942a624-a026-4fe7-86d2-dd4d5a907e6f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '72', 'date': 'Mon, 03 May 2021 15:14:59 GMT'}, 'RetryAttempts': 2}}


In [45]:
from sagemaker.session import production_variant

variant1 = production_variant(
    model_name=model_name_a,
    instance_type="ml.t2.medium",
    initial_instance_count=1,
    variant_name='VariantA',
    initial_weight=0.5)
                              
variant2 = production_variant(
    model_name=model_name_b,
    instance_type="ml.t2.medium",
    initial_instance_count=1,
    variant_name='VariantB',
    initial_weight=0.5)

In [46]:
sagemaker_session.endpoint_from_production_variants(
    name=endpoint_name,
    production_variants=[variant1, variant2]
)

-----------------!

'ab-endpoint'

In [47]:
runtime_sm_client = boto3.client('sagemaker-runtime')

In [48]:
body = "42"

In [49]:
from time import sleep

for _ in range(0,50):
    response = runtime_sm_client.invoke_endpoint(
                            EndpointName = endpoint_name,
                            ContentType  = 'text/csv',
                            Body         = body)
    
    variant = response['InvokedProductionVariant']
    prediction = response['Body'].read().decode("utf-8")

    print(variant + " - "+ prediction)
    sleep(1)

VariantA - 1233.18227456514
VariantB - 1233.1822745651407
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantB - 1233.1822745651407
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantB - 1233.1822745651407
VariantA - 1233.18227456514
VariantA - 1233.18227456514
VariantB - 1233.18227456

In [50]:
for _ in range(0,50):
    response = runtime_sm_client.invoke_endpoint(
                            EndpointName = endpoint_name,
                            ContentType  = 'text/csv',
                            TargetVariant='VariantB',
                            Body         = body)

    variant = response['InvokedProductionVariant']
    prediction = response['Body'].read().decode("utf-8")

    print(variant + " - "+ prediction)
    sleep(1)

VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB - 1233.1822745651407
VariantB -

In [51]:
response = sm_client.delete_endpoint(
    EndpointName=endpoint_name
)

response

{'ResponseMetadata': {'RequestId': 'e4d1826f-425b-4c35-86b0-bef1966dd161',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e4d1826f-425b-4c35-86b0-bef1966dd161',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 03 May 2021 15:25:15 GMT'},
  'RetryAttempts': 0}}